## Homework
### Question 1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:

```bash
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"
What's the version of dlt that you installed?
```

#### dlt Resourse

In [1]:
!pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import dlt

## A1

In [10]:
import requests

In [3]:
dlt.__version__

'1.13.0'

For reading the FAQ data, we have this helper function:

```bash
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc
```

Annotate it with @dlt.resource. We will use it when creating a dlt pipeline.

## Question 2. dlt pipeline

Now let's create a pipeline.

We need to define a destination for that. Let's use the qdrant one:

```bash
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)
```

In this case, we tell dlt (and Qdrant) to create a folder with our data, and the name for it will be db.qdrant

Let's run it:

```
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)
```

How many rows were inserted into the zoomcamp_data collection?

Look for `"Normalized data for the following tables:"` in the trace output.

In [5]:
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [7]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

In [11]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
#print(pipeline.last_trace)

In [14]:
print(pipeline.last_trace)

Run started at 2025-07-10 03:01:47.229220+00:00 and COMPLETED in 20.56 seconds with 4 steps.
Step extract COMPLETED in 1.31 seconds.

Load package 1752116508.1447442 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.09 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752116508.1447442 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 18.25 seconds.
Pipeline zoomcamp_pipeline load step completed in 18.23 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /app/db.qdrant location to store data
Load package 1752116508.1447442 is LOADED and contains no failed jobs

Step run COMPLETED in 20.56 seconds.
Pipeline zoomcamp_pipeline load step completed in 18.23 seconds
1 load package(s) were loaded to destination qdrant and into datas

A2.

Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)


## Question 3. Embeddings
When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the meta.json file created in the target folder.

In [17]:
import os
os.listdir()

['.ipynb_checkpoints',
 'db.qdrant',
 'docker-compose.yml',
 'Dockerfile',
 'hw3.ipynb',
 'qdrant_storage']

In [15]:
import json

In [22]:
with open("db.qdrant/meta.json", 'r', encoding='utf-8') as f_in:
    dict_ = json.load(f_in)

## A3

In [28]:
dict_['collections']['zoomcamp_tagged_data']['vectors']

{'fast-bge-small-en': {'size': 384,
  'distance': 'Cosine',
  'hnsw_config': None,
  'quantization_config': None,
  'on_disk': None,
  'datatype': None,
  'multivector_config': None}}